# Captum tests

Needed to modify the model code slightly, so the rest of the repo won't work

Don't forget to download the models from the README

In [ ]:
import argparse
import multiprocessing as mp
import os
import os.path as osp

import numpy as np
import torch
import yaml
from munch import Munch
from softgroup.data import build_dataloader, build_dataset
from softgroup.model import SoftGroup
from softgroup.util import (collect_results_cpu, get_dist_info, get_root_logger, init_dist,
                            is_main_process, load_checkpoint, rle_decode)
from torch.nn.parallel import DistributedDataParallel
from tqdm import tqdm

args = argparse.Namespace(
    config="./configs/softgroup/softgroup_s3dis_fold5.yaml",
    # Download and extract this
    checkpoint="./models/softgroup_s3dis_spconv2.pth",
    dist=False,
)

cfg_txt = open(args.config, 'r').read()
cfg = Munch.fromDict(yaml.safe_load(cfg_txt))
logger = get_root_logger()

model = SoftGroup(**cfg.model).cuda()
logger.info(f'Load state dict from {args.checkpoint}')
load_checkpoint(args.checkpoint, logger, model)

dataset = build_dataset(cfg.data.test, logger)
dataloader = build_dataloader(dataset, training=False, dist=args.dist, **cfg.dataloader.test)


In [ ]:
#model.__dict__

In [ ]:
with torch.no_grad():
    model.eval()
    for i, batch in tqdm(enumerate(dataloader), total=len(dataset)):
        # Set params here so the model only takes coords and feats as input
        #model.set_params(**batch)
        #result = model(batch['coords_float'], batch['feats'])
        #results.append(result)
        print(batch[ "scan_ids" ])
        # selected rooms: 4, 22
        #if i + 1 == 4:
        if batch[ "scan_ids" ] == ['Area_5_storage_2']:
            break

Input tensors

In [ ]:
print(batch.keys())
#print(batch['voxel_coords'].shape)
print(batch['coords_float'].shape)
print(batch['feats'].shape)
#batch['scan_ids']
#batch['batch_idxs']
#batch['voxel_coords']
#batch['p2v_map']
#batch['v2p_map']
#print(batch['coords_float'])
#print(batch['feats'])
#batch['semantic_labels']
#batch['instance_labels']
#batch['instance_pointnum']
#batch['instance_cls']
#batch['pt_offset_labels']
#batch['spatial_shape']
#batch['batch_size']

In [ ]:
# Create point cloud line by the points quantity
distance = 2.00
batch_coords_x = torch.arange(0, batch['coords_float'].shape[0] * distance, distance) # arithmetic progression of points
batch_coords_y = torch.full(batch_coords_x.shape, torch.mean(batch['coords_float'][:, 1]).item())
batch_coords_z = torch.full(batch_coords_x.shape, torch.mean(batch['coords_float'][:, 2]).item())
batch_coords_x = batch_coords_x.reshape(-1, 1)
batch_coords_y = batch_coords_y.reshape(-1, 1)
batch_coords_z = batch_coords_z.reshape(-1, 1)
batch_coords = torch.concat((batch_coords_x, batch_coords_y, batch_coords_z), 1)

Show point cloud

In [ ]:
#import nubilum_bk.utils.nubilum_utils as nb_utils
import nubilum.utils as nb_utils
def show_color_cloud(coords, colors):
    colors = (colors + 1)/2*255 # change color interval from [-1, 1] to [0, 255]
    nb_utils.show_point_cloud(coords, colors)
    #nb_utils.nubilum_utils.show_point_cloud(batch["coords_float"], colors)

In [ ]:
#show_color_cloud(batch_coords, batch['feats'])

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

model.train()
model.set_params(**batch)

# output always have this format: dict_keys(['scan_id', 'semantic_labels', 'instance_labels', 'coords_float', 'color_feats', 'semantic_preds', 'offset_preds', 'offset_labels', 'semantic_scores'])
# remember that the order of points in the output is different from the batch

out = model(batch_coords, batch['feats'])

instance_labels = torch.LongTensor(out['instance_labels'])

classes = {0: 'ceiling', 1: 'floor', 2: 'wall', 3: 'beam', 4: 'column', 5: 'window', 6: 'door', 7: 'chair', 8: 'table', 9: 'bookcase', 10: 'sofa', 11: 'board', 12: 'clutter'}

nb_utils.show_point_cloud_classification_plotly(out['coords_float'], out['semantic_preds'], instance_labels, classes_dict=classes)
#nb_utils.show_point_cloud_classification_k3d(batch['coords_float'], out['semantic_preds'])

# Model Wrappers

In [ ]:
#from nubilum_local_bk.forward.nubilum_forward import InstanceWrappedModel, PointWrappedModel, SummarizedWrappedModel
from nubilum.forward import InstanceWrappedModel, PointWrappedModel, SummarizedWrappedModel
        
instance_wrap = InstanceWrappedModel(model)
point_wrap = PointWrappedModel(model)
summarized_wrap = SummarizedWrappedModel(model)

# Saliency

In [ ]:
#from nubilum_local_bk.attr.nubilum_saliency import NubilumSaliency
from nubilum.attr import NubilumSaliency

model.train() # TODO: See if this works, if not, use model.train
model.set_params(**batch)

coords_float = batch['coords_float'].float().to("cuda:0").requires_grad_(True)
feats = batch['feats'].float().to("cuda:0").requires_grad_(True)

sl_instance = NubilumSaliency(instance_wrap)
sl_point = NubilumSaliency(point_wrap)
sl_summary = NubilumSaliency(summarized_wrap)

# Outputs
    0: 'ceiling',
    1: 'floor',
    2: 'wall',
    3: 'beam',
    4: 'column',
    5: 'window',
    6: 'door',
    7: 'chair',
    8: 'table',
    9: 'bookcase',
    10: 'sofa',
    11: 'board',
    12: 'clutter',

In [ ]:
# Point of interest on evidence in the Point Cloud

# REMEMBER TO USE ALWAYS THE POINTS INDICES FROM THE OUTPUT, THANKS SOFTGROUP FOR CHANGING THEM :DD

poi = 0
ioi = 16

ceiling = torch.tensor(0, device="cuda")
floor = torch.tensor(1, device="cuda")
wall = torch.tensor(2, device="cuda")
windows = torch.tensor(5, device="cuda")
doors = torch.tensor(6, device="cuda")
chairs = torch.tensor(7, device="cuda") # remember to always change this when changing the analysis
tables = torch.tensor(8, device="cuda")
bookcases = torch.tensor(9, device="cuda")
clutter = torch.tensor(12, device="cuda")
#nb_utils.show_poi(poi, out["coords_float"])

In [ ]:
def log_scale_attributes(attributes: np.array, signed=False):
    log_attr = np.log10(np.abs(attributes) + 1e-10)
    if signed: # to verify the positive and negative contributions
        signs = np.sign(attributes)
        signed_log_attr = signs * (log_attr + abs(np.min(log_attr))) # It reverts the abs procedure done to calculate the log before
        return signed_log_attr
    else:
        return log_attr

# Attributes

## Instance Wrapper

In [ ]:
instance_wrap_attributes = sl_instance.attribute(inputs=(coords_float, feats), target=chairs, additional_forward_args=(out['instance_labels'], ioi, 'semantic_scores'))
inst_point_attrs = nb_utils.sum_point_attributes(instance_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(inst_point_attrs, batch['coords_float'], "All attributes")


In [ ]:
instance_wrap_attributes

In [ ]:
# Chair importance ratio
instance_mask = batch['instance_labels'] == ioi
chair_attr = inst_point_attrs[instance_mask]

ratio = chair_attr.sum() / inst_point_attrs.sum()
print(ratio)

## Point Wrapper

In [ ]:
batch_coords = batch_coords.to('cuda:0')
point_wrap_attributes = sl_point.attribute((batch_coords, feats), target = ceiling, additional_forward_args=(poi, 'semantic_scores')) # coords were changed for rf test
point_point_attrs = nb_utils.sum_point_attributes(point_wrap_attributes)
point_log_attr = log_scale_attributes(point_point_attrs.cpu().detach().numpy())
print(point_log_attr)
nb_utils.explain_plotly(torch.tensor(point_log_attr), batch_coords.cpu())
#nb_utils.explain_k3d(torch.tensor(point_log_attr), batch_coords.cpu(), "All attributes")

## Summary Wrapper

In [ ]:
summary_wrap_attributes = sl_summary.attribute((coords_float, feats), target = bookcases, additional_forward_args=('semantic_scores'))
summ_point_attrs = nb_utils.sum_point_attributes(summary_wrap_attributes)
#nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])
nb_utils.explain_k3d(summ_point_attrs, batch['coords_float'], "All attributes")

Sempre que se estiver usando o termo "**atributo**", estamos falando da importância ou score de relevância do input para um determinado output obtido através de um modelo.

In [ ]:
inst_point_attrs = nb_utils.sum_point_attributes(instance_wrap_attributes)
point_point_attrs = nb_utils.sum_point_attributes(point_wrap_attributes)
summ_point_attrs = nb_utils.sum_point_attributes(summary_wrap_attributes)

# Show attribution

In [ ]:
# Instance
nb_utils.explain_plotly(inst_point_attrs, batch['coords_float'])

In [ ]:
nb_utils.explain_k3d(inst_point_attrs, batch['coords_float'], "All attributes")